In [ ]:
#import sys
#!{sys.executable} -m conda install psycopg2-binary

In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
password = input("Password for MSiA: ")
conn = psycopg2.connect(
    host="pg.analytics.northwestern.edu",
    database="2022-everything-team08",
    user="zzp8676", #change here
    password=password)

print("Connected.")
cur = conn.cursor()

Password for MSiA:  Zz007270zy!@


Connected.


In [ ]:
# Query the database and obtain data as Python objects
#cur.execute('SELECT * FROM public.\"DEPTINFO\";')
#cur.fetchone()


In [65]:
import pandas.io.sql as psql
import random

In [78]:
#df = psql.read_sql('SELECT * FROM msia400.trnsact WHERE orgprice > amt AND extract(year from saledate) = 2005 limit 100', conn)

random.seed(77)

df = psql.read_sql('SELECT * FROM msia400.trnsact tr INNER JOIN msia400.skstinfo sk ON sk.sku = tr.store AND sk.store = tr.register LIMIT 10000', conn)

df.head()


,store,register,seq,trannum,interid,saledate,stype,quantity,orgprice,amt,amt2,sku,mic,label,sku,store,cost,retail,label
0,19171,9504,250,1500,000000000,2005-04-05,P,1,16.5,16.5,16.5,363500067,281,0,19171,9504,11.0,18.0,0
1,19171,9504,570,300,000000000,2005-05-06,P,1,16.5,16.5,16.5,675100084,281,0,19171,9504,11.0,18.0,0
2,19171,9504,570,400,000000000,2004-12-06,P,1,16.5,16.5,16.5,364900098,281,0,19171,9504,11.0,18.0,0
3,19171,9504,570,600,000000000,2004-08-10,P,1,16.5,16.5,16.5,760200076,281,0,19171,9504,11.0,18.0,0
4,19171,9504,570,800,000000000,2004-12-17,P,1,16.5,16.5,16.5,896900179,281,0,19171,9504,11.0,18.0,0


In [79]:
from datetime import datetime
import pandas as pd

# Keep rows with orgprice > 0
df[df['amt'] <= df['orgprice']]

# Drop rows with null values in any of the three columns
df.dropna(subset = ['saledate','retail','orgprice'], inplace = True)

h = df['saledate'].astype(str).str.split("-")

month = [h[i][1] for i in range(10000)]
df['month'] = month

pd.get_dummies(df, columns = ['month'])


,store,register,seq,trannum,interid,saledate,stype,quantity,orgprice,amt,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,19171,9504,250,1500,000000000,2005-04-05,P,1,16.5,16.5,...,0,1,0,0,0,0,0,0,0,0
1,19171,9504,570,300,000000000,2005-05-06,P,1,16.5,16.5,...,0,0,1,0,0,0,0,0,0,0
2,19171,9504,570,400,000000000,2004-12-06,P,1,16.5,16.5,...,0,0,0,0,0,0,0,0,0,1
3,19171,9504,570,600,000000000,2004-08-10,P,1,16.5,16.5,...,0,0,0,0,0,1,0,0,0,0
4,19171,9504,570,800,000000000,2004-12-17,P,1,16.5,16.5,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19633,3304,230,2000,640600557,2004-09-06,P,1,21.0,21.0,...,0,0,0,0,0,0,1,0,0,0
9996,19633,3304,230,2000,000000000,2004-12-31,P,1,21.0,21.0,...,0,0,0,0,0,0,0,0,0,1
9997,19633,3304,230,2000,723400992,2005-08-23,P,1,22.5,22.5,...,0,0,0,0,0,1,0,0,0,0
9998,19633,3304,230,2100,000000000,2005-02-05,P,1,21.0,21.0,...,0,0,0,0,0,0,0,0,0,0


df = pd.read_csv("deptinfo.csv")
df.to_sql(‘test_schema.dept_python_import’, con=conn, if_exists=’replace’, index=False)


In [ ]:
# # Make the changes to the database persistent
# conn.commit()
# Close communication with the database
cur.close()
conn.close()